In [1]:
from pprint import pprint
import copy

i = 0
group = {}
transactions = {}

def reset():
    global i
    i = 0
    createGroup()

def addPerson(name):
    group['balances'][name] = {}

    # link people
    for p in group['people']:
        group['balances'][name][p] = 0
        group['balances'][p][name] = 0

    group['ledger'][name] = 0
    # add person to group
    group['people'][name] = {'name': name}

def createGroup():
    # clear shit
    group.clear()
    transactions.clear()

    # setup group
    group['ledger'] = {}
    group['balances'] = {}
    group['people'] = {}
    group['transactions'] = {}
    addPerson("person1")
    addPerson("person2")
    addPerson("person3")

createGroup()
pprint(group)

{'balances': {'person1': {'person2': 0, 'person3': 0},
              'person2': {'person1': 0, 'person3': 0},
              'person3': {'person1': 0, 'person2': 0}},
 'ledger': {'person1': 0, 'person2': 0, 'person3': 0},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {}}


In [103]:
def add_transaction(who_got_how_much, who_paid_how_much):
    global i
    paid_total = sum(who_paid_how_much.values())
    used_total = sum(who_got_how_much.values())
    assert paid_total == used_total

    ledger = {}
    people_involved = set(who_got_how_much.keys()).union(set(who_paid_how_much.keys()))
    for p in people_involved:
        # update ledger
        used, paid = 0, 0
        if p in who_got_how_much:
            used = who_got_how_much[p]
        if p in who_paid_how_much:
            paid = who_paid_how_much[p]
        ledger[p] = paid - used

    # update ledger
    for k, v in ledger.items():
        group['ledger'][k] += v

    transaction = {'id': i, 'who_used': who_got_how_much.copy(), 'who_paid': who_paid_how_much.copy()}
    # 2,1,1,2
    for p_paid in who_paid_how_much.keys():
        paid = ledger[p_paid]
        # calculate how much if any this person has pair
        if p_paid in who_got_how_much:
            paid -= who_got_how_much[p_paid]
        for p_used in who_got_how_much.keys():
            # if the person that paid is not the same as the person that used
            if p_paid != p_used:
                if ledger[p_used] < 0 :
                    # if the amount paid is greater than the amount used
                    if ledger[p_used] + paid > 0:
                        group['balances'][p_paid][p_used] -= ledger[p_used]
                        group['balances'][p_used][p_paid] += ledger[p_used]
                    # if the amount paid is equal to or less than the amount used
                    else:
                        paid = paid if paid > 0 else -paid
                        group['balances'][p_paid][p_used] += paid
                        group['balances'][p_used][p_paid] -= paid

            # # if the person that paid is not the same as the person that used
            # if p_paid != p_used:
            #     if paid > 0:
            #         if ledger[p_used] < 0 :
            #             # if the amount paid is greater than the amount used
            #             if ledger[p_used] + paid > 0:
            #                 group['balances'][p_paid][p_used] -= ledger[p_used]
            #                 group['balances'][p_used][p_paid] += ledger[p_used]
            #             # if the amount paid is equal to or less than the amount used
            #             else:
            #                 group['balances'][p_paid][p_used] += paid
            #                 group['balances'][p_used][p_paid] -= paid
            #     else:
            #         if ledger[p_used] < 0 :
            #             # if the amount paid is greater than the amount used
            #             if ledger[p_used] + paid > 0:
            #                 group['balances'][p_paid][p_used] -= ledger[p_used]
            #                 group['balances'][p_used][p_paid] += ledger[p_used]
            #             # if the amount paid is equal to or less than the amount used
            #             else:
            #                 group['balances'][p_paid][p_used] -= paid
            #                 group['balances'][p_used][p_paid] += paid

    # save transaction
    group['transactions'][i] = transaction
    i += 1
    pprint(group)

    return transaction




In [105]:
reset()
t1 = add_transaction(who_got_how_much={
    'person1': 20,
}, who_paid_how_much={
    'person1': 20,
})


{'balances': {'person1': {'person2': 0, 'person3': 0},
              'person2': {'person1': 0, 'person3': 0},
              'person3': {'person1': 0, 'person2': 0}},
 'ledger': {'person1': 0, 'person2': 0, 'person3': 0},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {0: {'id': 0,
                      'who_paid': {'person1': 20},
                      'who_used': {'person1': 20}}}}


In [106]:
t2 = add_transaction(who_got_how_much={
    'person1': 20,
}, who_paid_how_much={
    'person2': 20,
})

{'balances': {'person1': {'person2': -20, 'person3': 0},
              'person2': {'person1': 20, 'person3': 0},
              'person3': {'person1': 0, 'person2': 0}},
 'ledger': {'person1': -20, 'person2': 20, 'person3': 0},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {0: {'id': 0,
                      'who_paid': {'person1': 20},
                      'who_used': {'person1': 20}},
                  1: {'id': 1,
                      'who_paid': {'person2': 20},
                      'who_used': {'person1': 20}}}}


In [107]:
t2 = add_transaction(who_got_how_much={
    'person2': 20,
}, who_paid_how_much={
    'person1': 20,
})

{'balances': {'person1': {'person2': 0, 'person3': 0},
              'person2': {'person1': 0, 'person3': 0},
              'person3': {'person1': 0, 'person2': 0}},
 'ledger': {'person1': 0, 'person2': 0, 'person3': 0},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {0: {'id': 0,
                      'who_paid': {'person1': 20},
                      'who_used': {'person1': 20}},
                  1: {'id': 1,
                      'who_paid': {'person2': 20},
                      'who_used': {'person1': 20}},
                  2: {'id': 2,
                      'who_paid': {'person1': 20},
                      'who_used': {'person2': 20}}}}


In [108]:
t3 = add_transaction(who_got_how_much={
    'person1': 20,
}, who_paid_how_much={
    'person2': 10,
    'person3': 10,
})

{'balances': {'person1': {'person2': -10, 'person3': -10},
              'person2': {'person1': 10, 'person3': 0},
              'person3': {'person1': 10, 'person2': 0}},
 'ledger': {'person1': -20, 'person2': 10, 'person3': 10},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {0: {'id': 0,
                      'who_paid': {'person1': 20},
                      'who_used': {'person1': 20}},
                  1: {'id': 1,
                      'who_paid': {'person2': 20},
                      'who_used': {'person1': 20}},
                  2: {'id': 2,
                      'who_paid': {'person1': 20},
                      'who_used': {'person2': 20}},
                  3: {'id': 3,
                      'who_paid': {'person2': 10, 'person3': 10},
                      'who_used': {'person1': 20}}}}


In [109]:
t4 = add_transaction(who_got_how_much={
    'person2': 10,
    'person3': 10,
}, who_paid_how_much={
    'person1': 20,
})

{'balances': {'person1': {'person2': 0, 'person3': 0},
              'person2': {'person1': 0, 'person3': 0},
              'person3': {'person1': 0, 'person2': 0}},
 'ledger': {'person1': 0, 'person2': 0, 'person3': 0},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {0: {'id': 0,
                      'who_paid': {'person1': 20},
                      'who_used': {'person1': 20}},
                  1: {'id': 1,
                      'who_paid': {'person2': 20},
                      'who_used': {'person1': 20}},
                  2: {'id': 2,
                      'who_paid': {'person1': 20},
                      'who_used': {'person2': 20}},
                  3: {'id': 3,
                      'who_paid': {'person2': 10, 'person3': 10},
                      'who_used': {'person1': 20}},
                  4: {'id': 4,
                      'who_paid': {'person1': 20},
             

In [110]:
t5 = add_transaction(who_got_how_much={
    'person2': 30,
}, who_paid_how_much={
    'person2': 30,
})

{'balances': {'person1': {'person2': 0, 'person3': 0},
              'person2': {'person1': 0, 'person3': 0},
              'person3': {'person1': 0, 'person2': 0}},
 'ledger': {'person1': 0, 'person2': 0, 'person3': 0},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {0: {'id': 0,
                      'who_paid': {'person1': 20},
                      'who_used': {'person1': 20}},
                  1: {'id': 1,
                      'who_paid': {'person2': 20},
                      'who_used': {'person1': 20}},
                  2: {'id': 2,
                      'who_paid': {'person1': 20},
                      'who_used': {'person2': 20}},
                  3: {'id': 3,
                      'who_paid': {'person2': 10, 'person3': 10},
                      'who_used': {'person1': 20}},
                  4: {'id': 4,
                      'who_paid': {'person1': 20},
             

In [111]:
t5 = add_transaction(who_got_how_much={
    'person2': 30,
    'person3': 30,
}, who_paid_how_much={
    'person2': 30,
    'person3': 30,
})

{'balances': {'person1': {'person2': 0, 'person3': 0},
              'person2': {'person1': 0, 'person3': 0},
              'person3': {'person1': 0, 'person2': 0}},
 'ledger': {'person1': 0, 'person2': 0, 'person3': 0},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {0: {'id': 0,
                      'who_paid': {'person1': 20},
                      'who_used': {'person1': 20}},
                  1: {'id': 1,
                      'who_paid': {'person2': 20},
                      'who_used': {'person1': 20}},
                  2: {'id': 2,
                      'who_paid': {'person1': 20},
                      'who_used': {'person2': 20}},
                  3: {'id': 3,
                      'who_paid': {'person2': 10, 'person3': 10},
                      'who_used': {'person1': 20}},
                  4: {'id': 4,
                      'who_paid': {'person1': 20},
             

In [112]:
t6 = add_transaction(who_got_how_much={
    'person2': 20,
    'person3': 40,
}, who_paid_how_much={
    'person2': 30,
    'person3': 30,
})

{'balances': {'person1': {'person2': 0, 'person3': 0},
              'person2': {'person1': 0, 'person3': 10},
              'person3': {'person1': 0, 'person2': -10}},
 'ledger': {'person1': 0, 'person2': 10, 'person3': -10},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {0: {'id': 0,
                      'who_paid': {'person1': 20},
                      'who_used': {'person1': 20}},
                  1: {'id': 1,
                      'who_paid': {'person2': 20},
                      'who_used': {'person1': 20}},
                  2: {'id': 2,
                      'who_paid': {'person1': 20},
                      'who_used': {'person2': 20}},
                  3: {'id': 3,
                      'who_paid': {'person2': 10, 'person3': 10},
                      'who_used': {'person1': 20}},
                  4: {'id': 4,
                      'who_paid': {'person1': 20},
       

In [113]:
t6 = add_transaction(who_got_how_much={
    'person2': 40,
    'person3': 20,
}, who_paid_how_much={
    'person2': 30,
    'person3': 30,
})

{'balances': {'person1': {'person2': 0, 'person3': 0},
              'person2': {'person1': 0, 'person3': 0},
              'person3': {'person1': 0, 'person2': 0}},
 'ledger': {'person1': 0, 'person2': 0, 'person3': 0},
 'people': {'person1': {'name': 'person1'},
            'person2': {'name': 'person2'},
            'person3': {'name': 'person3'}},
 'transactions': {0: {'id': 0,
                      'who_paid': {'person1': 20},
                      'who_used': {'person1': 20}},
                  1: {'id': 1,
                      'who_paid': {'person2': 20},
                      'who_used': {'person1': 20}},
                  2: {'id': 2,
                      'who_paid': {'person1': 20},
                      'who_used': {'person2': 20}},
                  3: {'id': 3,
                      'who_paid': {'person2': 10, 'person3': 10},
                      'who_used': {'person1': 20}},
                  4: {'id': 4,
                      'who_paid': {'person1': 20},
             